In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
df = pd.read_csv('../data/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv', parse_dates=['collection_week'])
df_ny = df[df.state=='NY']

/var/folders/4n/fnm27kds4tsbfbt5m1y08tp80000gq/T/ipykernel_1047/3698467594.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv', parse_dates=['collection_week'])


In [3]:
df_l = pd.read_csv('../data/NY/New_York_State_Statewide_COVID-19_Hospitalizations_and_Beds.csv', parse_dates=['As of Date'])
df_l.head()

,As of Date,Facility PFI,Facility Name,DOH Region,Facility County,Facility Network,NY Forward Region,Patients Currently Hospitalized,Patients Newly Admitted,Patients Positive After Admission,...,Patients Age Less Than 1 Year,Patients Age 1 To 4 Years,Patients Age 5 to 19 Years,Patients Age 20 to 44 Years,Patients Age 45 to 54 Years,Patients Age 55 to 64 Years,Patients Age 65 to 74 Years,Patients Age 75 to 84 Years,Patients Age Greater Than 85 Years,Hospitalized Indicator
0,2020-03-26,1,ALBANY MEDICAL CENTER HOSPITAL,CAPITAL DISTRICT REGIONAL OFFICE,ALBANY,ALBANY MEDICAL CENTER,CAPITAL REGION,12.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,4.0,3.0,2.0,1.0,0
1,2020-03-26,5,ST PETERS HOSPITAL,CAPITAL DISTRICT REGIONAL OFFICE,ALBANY,ST. PETERS HEALTH PARTNERS,CAPITAL REGION,10.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,1.0,0
2,2020-03-26,37,CUBA MEMORIAL HOSPITAL INC,WESTERN REGIONAL OFFICE,ALLEGANY,INDEPENDENT,WESTERN NEW YORK,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2020-03-26,39,MEMORIAL HOSP OF WM F AND GERTRUDE F JONES AKA...,WESTERN REGIONAL OFFICE,ALLEGANY,UNIVERSITY OF ROCHESTER MEDICAL CENTER,WESTERN NEW YORK,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2020-03-26,42,UNITED HEALTH SERVICES HOSPITALS INC - BINGHAM...,CENTRAL REGIONAL OFFICE,BROOME,"UNITED HEALTH SERVICES HOSPITALS, INC.",SOUTHERN TIER,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
luisa_hsp = df_l['Facility Name'].unique()
gonick_hsp = df_ny['hospital_name'].unique()
len(luisa_hsp), len(gonick_hsp)

(199, 166)

In [7]:
missing = [g for g in gonick_hsp if g not in luisa_hsp]
len(missing), missing 

(74,
 ['CALVARY HOSPITAL INC',
  'THE BURDETT CARE CENTER',
  'ROME MEMORIAL HOSPITAL, INC',
  'CUBA MEMORIAL HOSPITAL, INC',
  'AURELIA OSBORN FOX MEMORIAL HOSPITAL',
  'NYACK HOSPITAL',
  'HUDSON VALLEY HOSPITAL CENTER',
  'ELIZABETHTOWN COMMUNITY HOSPITAL',
  "MOUNT ST MARY'S HOSPITAL AND  HEALTH CENTER",
  'CLAXTON-HEPBURN MEDICAL CENTER',
  'JONES MEMORIAL HOSPITAL',
  'GARNET HEALTH  MEDICAL CENTER CATSKILLS - (CSK)',
  'MONTEFIORE MEDICAL CENTER',
  'CONEY ISLAND HOSPITAL CENTER',
  'GARNET HEALTH MEDICAL CENTER',
  "O'CONNOR HOSPITAL",
  'GUTHRIE CORTLAND REGIONAL MEDICAL CENTER',
  'LINCOLN MEDICAL & MENTAL HEALTH CENTER',
  'UNIVERSITY HOSPITAL S U N Y HEALTH SCIENCE CENTER',
  "FAXTON-ST LUKE'S HEALTHCARE",
  'RIVER HOSPITAL CLINICS',
  'HEALTHALLIANCE HOSPITAL MARYS AVENUE CAMPUS',
  'IRA DAVENPORT MEMORIAL HOSPITAL',
  'UNITY HOSPITAL OF ROCHESTER',
  'UNITED MEMORIAL MEDICAL CENTER',
  "ST JOSEPH'S MEDICAL CENTER",
  "ST PETER'S HOSPITAL",
  'WOODHULL MEDICAL & MENTAL HEA

In [33]:
missing = [l for l in luisa_hsp if l not in gonick_hsp]
len(missing), missing 

(107,
 ['ST PETERS HOSPITAL',
  'CUBA MEMORIAL HOSPITAL INC',
  'MEMORIAL HOSP OF WM F AND GERTRUDE F JONES AKA JONES MEMORIAL HOSP',
  'UNITED HEALTH SERVICES HOSPITALS INC - BINGHAMTON GENERAL HOSPITAL',
  'OUR LADY OF LOURDES MEMORIAL HOSPITAL INC',
  'UNITED HEALTH SERVICES HOSPITAL, INC. - WILSON MEDICAL CENTER',
  'AUBURN MEMORIAL HOSPITAL',
  'BROOKS-TLC HOSPITAL SYSTEM, INC. (DUNKIRK)',
  'WESTFIELD MEMORIAL HOSPITAL INC',
  'ST. JOSEPHS HOSPITAL',
  'CHENANGO MEMORIAL HOSPITAL INC',
  'THE UNIVERSITY OF VERMONT HEALTH NETWORK-CHAMPLAIN VALLEY PHYSICIANS HOSPITAL',
  'GUTHRIE CORTLAND MEDICAL CENTER',
  'OCONNOR HOSPITAL',
  'MARGARETVILLE HOSPITAL',
  'DELAWARE VALLEY HOSPITAL INC',
  'MID-HUDSON VALLEY DIVISION OF WESTCHESTER MEDICAL CENTER',
  'BUFFALO GENERAL MEDICAL CENTER',
  'JOHN R. OISHEI CHILDRENS HOSPITAL',
  'MERCY HOSPITAL',
  'UNIVERSITY HOSPITAL',
  'SISTERS OF CHARITY HOSPITAL - ST JOSEPH CAMPUS',
  'THE UNIVERSITY OF VERMONT HEALTH NETWORK - ELIZABETHTOWN COMMU

In [5]:
import difflib

In [9]:
df = pd.DataFrame({'hsp_luisa': luisa_hsp})
df.head()

,hsp_luisa
0,ALBANY MEDICAL CENTER HOSPITAL
1,ST PETERS HOSPITAL
2,CUBA MEMORIAL HOSPITAL INC
3,MEMORIAL HOSP OF WM F AND GERTRUDE F JONES AKA...
4,UNITED HEALTH SERVICES HOSPITALS INC - BINGHAM...


In [14]:
df['hsp_gonick']=gonick_hsp + (['']*(len(luisa_hsp)-len(gonick_hsp)))

ValueError: operands could not be broadcast together with shapes (166,) (32,) 

In [54]:
df['col1'] = [(len(difflib.get_close_matches(x, gonick_hsp , cutoff=0.5))>=1)*1 
              for x in df['hsp_luisa']]

In [66]:
cols = []
for hsp in df['hsp_luisa']:
    similar_hsps = difflib.get_close_matches(hsp, gonick_hsp , cutoff=0.5)
    if len(similar_hsps):
        cols.append(similar_hsps[0])
    else:
        cols.append(0)    

df['col1'] = cols

In [71]:
df[df.col1==0]

,hsp_luisa,col1
3,MEMORIAL HOSP OF WM F AND GERTRUDE F JONES AKA...,0
130,SJRH - DOBBS FERRY PAVILION,0
185,MONTEFIORE MED CTR - JACK D WEILER HOSP OF A E...,0
188,SOUTH NASSAU COMMUNITIES HOSPITAL OFF-CAMPUS E...,0
189,LENOX HEALTH GREENWICH VILLAGE,0
195,NYC H+H - BILLIE JEAN KING TENNIS CENTER,0


In [68]:
difflib.get_close_matches('LENOX', gonick_hsp , cutoff=0.1)

['LENOX HILL HOSPITAL', 'GLEN COVE HOSPITAL', 'HELEN HAYES HOSPITAL']

In [56]:
gonick_hsp.tolist().index('SJRH - DOBBS FERRY PAVILION')

ValueError: 'SJRH - DOBBS FERRY PAVILION' is not in list

In [72]:
unmapped = [g for g in gonick_hsp if g not in df['col1'].values]
len(unmapped), unmapped

(24,
 ['CALVARY HOSPITAL INC',
  'THE BURDETT CARE CENTER',
  'AURELIA OSBORN FOX MEMORIAL HOSPITAL',
  'NYACK HOSPITAL',
  'HUDSON VALLEY HOSPITAL CENTER',
  'ELIZABETHTOWN COMMUNITY HOSPITAL',
  'CLAXTON-HEPBURN MEDICAL CENTER',
  'GARNET HEALTH  MEDICAL CENTER CATSKILLS - (CSK)',
  'GARNET HEALTH MEDICAL CENTER',
  'HEALTHALLIANCE HOSPITAL MARYS AVENUE CAMPUS',
  'AUBURN  COMMUNITY  HOSPITAL',
  'KALEIDA HEALTH',
  'ALICE HYDE MEDICAL CENTER',
  'SUNNYVIEW HOSPITAL AND REHABILITATION CENTER',
  'UNITY SPECIALTY HOSPITAL',
  'GARNET HEALTH MEDICAL CENTER-CATSKILLS(GMHERMANN)',
  'SUNY HEALTH SCIENCE CENTER AT BROOKLYN UNIVERSITY',
  'N Y EYE AND EAR INFIRMARY',
  'MONROE COMMUNITY HOSPITAL',
  'ROSWELL PARK CANCER INSTITUTE',
  'ST BARNABAS HOSPITAL',
  'HENRY J CARTER SPECIALTY HOSPITAL',
  'MERCY HOSPITAL OF BUFFALO',
  'TERENCE CARDINAL COOKE HEALTH CARE CENTER'])

In [73]:
df.to_csv('mapped_hsp.csv')